# k-scheduling tests

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from random import sample, seed

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, assemble
from qiskit.algorithms import amplitude_estimators, EstimationProblem
from qiskit.algorithms import IterativeAmplitudeEstimation as BaseIterativeAmplitudeEstimation
# from qiskit.extensions import UnitaryGate

from algorithms import IterativeAmplitudeEstimation, MaximumLikelihoodAmplitudeEstimation, FasterAmplitudeEstimation
from operators import *

In [2]:
n = 4
N = 2**n
k = N//2
marked = sample(range(N), k)

In [3]:
# Define the estimation problem and oracle function
def good_state(state):
    bin_marked = [(n-len(bin(s))+2)*'0'+bin(s)[2:] for s in marked]
    return (state in bin_marked)

problem = EstimationProblem(
    state_preparation=A(n),  # A operator
    grover_operator=Q(n, marked),  # Q operator
    objective_qubits=range(n),
    is_good_state=good_state  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

problem2 = EstimationProblem(
    state_preparation=A(n),  # A operator
    grover_operator=Q(n, marked),  # Q operator
    objective_qubits=range(n),
    is_good_state=good_state  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

In [4]:
# use local simulator
aer_sim = Aer.get_backend('aer_simulator')
shots = 100

## IQAE: attempt exponential decay in the number of shots per iteration to account for deeper circuits.

In [5]:
# TODO: package this in a function to make it nicer / generalize

# n_trials = 100
# max_depth_log2 = 5
# queries = np.zeros((max_depth_log2-1, n_trials))
# errs = np.zeros((max_depth_log2-1, n_trials))

# for i in range(2,max_depth_log2+1):
#     for j in range(n_trials):
#         mlae = MLAE(range(i), quantum_instance=aer_sim)
#         result = mlae.estimate(problem)
#         queries[i-2,j] = result.num_oracle_queries
#         errs[i-2,j] = abs(k/N - result.estimation)

In [6]:
# parameters for IQAE
alpha = 0.05
confint_method = 'chernoff'

In [7]:
powers = range(10, 20)
epsilons = [(1.0 / (2**power)) for power in powers]
iae_results = [] 
biae_results = []

for i, epsilon in enumerate(epsilons):
    IAE = IterativeAmplitudeEstimation(epsilon_target=epsilon, alpha=alpha, confint_method=confint_method, quantum_instance=aer_sim)
    BIAE = BaseIterativeAmplitudeEstimation(epsilon_target=epsilon, alpha=alpha, confint_method=confint_method, quantum_instance=aer_sim)
    iae_result = IAE.estimate(problem)
    biae_result = BIAE.estimate(problem2)
    iae_results.append(iae_result)
    biae_results.append(biae_result)

0 None
prev_k/k = 1
nshots: 1024
2 0
prev_k/k = 1
nshots: 1024
12 2
prev_k/k = 0.16666666666666666
nshots: 171
12 12
prev_k/k = 1.0
nshots: 171
31 12
prev_k/k = 0.3870967741935484
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
prev_k/k = 1.0
nshots: 66
31 31
pre

KeyboardInterrupt: 

In [ ]:
IAE_RESULT = iae_results
BIAE_RESULT = biae_results

In [ ]:
iae_epsilon = [(res.confidence_interval_processed[1] - res.confidence_interval_processed[0]) / 2 for res in IAE_RESULT]
iae_nshots  = [res.num_oracle_queries for res in IAE_RESULT]

biae_epsilon = [(res.confidence_interval_processed[1] - res.confidence_interval_processed[0]) / 2 for res in BIAE_RESULT]
biae_nshots  = [res.num_oracle_queries for res in BIAE_RESULT]

In [ ]:
fig = plt.figure()
ax = plt.gca()
ax.scatter(iae_nshots, iae_epsilon)
ax.plot(iae_nshots, iae_epsilon)
ax.scatter(biae_nshots, biae_epsilon)
ax.plot(biae_nshots, biae_epsilon)
ax.legend(['IAE', 'Base IAE'])
ax.set_yscale('log')
ax.set_xscale('log')
plt.xlim(1, 10**10)
plt.show()

In [ ]:
errors = np.array([abs(res.estimation - 1/2) for res in IAE_RESULT])

In [ ]:
plt.plot(iae_nshots, errors)
plt.plot(iae_nshots, iae_epsilon)
plt.show()